In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import percentileofscore
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
from matplotlib.lines import Line2D
import matplotlib as mpl
from matplotlib import font_manager
mpl.rcParams.update(mpl.rcParamsDefault)

# Define the scope and load credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials_path = "C:/Users/benoi/OneDrive/Desktop/bea/json credentials/bea-data-7dda3770b44f.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)

# Authenticate and connect to Google Sheets
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OfgrNbgZjwMMAuVdIhOLywXUHEw1xHzYaN8ZU2awsSs/edit?gid=1169755047')
worksheet = sheet.get_worksheet(0)
data = pd.DataFrame(worksheet.get_all_records())

def generate_cover_page(df, athlete_id, pdf):
    # Get athlete details
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        athlete_name = f"Athlete {athlete_id}"
        status = "N/A"
        dob = "N/A"
        level = "N/A"
        school_org = "N/A"
        grad_year = "N/A"
        position = "N/A"
        weight = "N/A"
        height = "N/A"
    else:
        athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
        status = athlete_data['Status'].iloc[0]
        dob = athlete_data['DOB'].iloc[0] if 'DOB' in athlete_data.columns else "N/A"
        level = athlete_data['Level'].iloc[0] if 'Level' in athlete_data.columns else "N/A"
        school_org = athlete_data['School/Org'].iloc[0] if 'School/Org' in athlete_data.columns else "N/A"
        grad_year = athlete_data['Grad Year'].iloc[0] if 'Grad Year' in athlete_data.columns else "N/A"
        position = athlete_data['Position'].iloc[0] if 'Position' in athlete_data.columns else "N/A"
        
        # Get the most recent Weigh-in data
        weigh_in_data = athlete_data[athlete_data['Test Type'] == 'Weigh-in']
        if not weigh_in_data.empty:
            weight_data = weigh_in_data[weigh_in_data['Test Sub-Type'] == 'Weigh-in']
            height_data = weigh_in_data[weigh_in_data['Test Sub-Type'] == 'Height']
            weight = weight_data['Weight'].iloc[-1] if not weight_data.empty else "N/A"
            height = height_data['Height'].iloc[-1] if not height_data.empty else "N/A"
        else:
            weight = "N/A"
            height = "N/A"

    # Get the current date
    created_date = datetime.now().strftime("%B %d, %Y")

    # Create the figure for the cover page
    fig, ax = plt.subplots(figsize=(8.5, 11))  # Letter size
    ax.axis('off')  # Remove axes for a clean design

    # Title
    ax.text(
        0.5, 0.85, "Beimel Elite Athletics\nSCOPE Report",
        fontsize=30, fontweight='bold', ha='center', va='center'
    )

    # Acronym breakdown aligned vertically on the left
    acronym_text = (
        "S.trength\n"
        "C.onsistency\n"
        "O.ptimization\n"
        "P.reparation\n"
        "E.ffort"
    )
    ax.text(
        0.4, 0.65, acronym_text,
        fontsize=14, fontweight='regular', ha='left', va='center', color="gray"
    )

    # Athlete details
    details_text = (
        f"Athlete: {athlete_name}\n"
        f"Membership: {status}\n"
        f"DOB: {dob}    H/W: {height}\"/{weight}lbs \n"
        f"{school_org}  {grad_year}\n"
        f"Position: {position}\n"
        f"Created Date: {created_date}"
    )
    ax.text(
        0.5, 0.4, details_text,
        fontsize=12, fontweight='regular', ha='center', va='center'
    )

    # Footer with subtle design
    ax.text(
        0.5, 0.1, "BEA Performance Analytics",
        fontsize=10, ha='center', va='center', color="gray", alpha=0.7
    )

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import percentileofscore

# Modular group-based metric definition
metric_groups = {
    'Strength': [
        ('Weigh-in', 'Height', ['Height']),
        ('Weigh-in', 'Weigh-in', ['Weight']),
        ('Back Squat', 'Starting Strength', ['Weight', 'Speed']),
        ('Back Squat', 'Strength-Speed', ['Weight', 'Speed']),
        ('Back Squat', 'Accelerative Strength', ['Weight', 'Speed']),
        ('Back Squat', 'Absolute Strength', ['Weight', 'Speed']),
        ('Bench Press', 'Starting Strength', ['Weight', 'Speed']),
        ('Bench Press', 'Strength-Speed', ['Weight', 'Speed']),
        ('Bench Press', 'Accelerative Strength', ['Weight', 'Speed']),
        ('Bench Press', 'Absolute Strength', ['Weight', 'Speed']),
        ('Deadlift', 'Starting Strength', ['Weight', 'Speed']),
        ('Deadlift', 'Strength-Speed', ['Weight', 'Speed']),
        ('Deadlift', 'Accelerative Strength', ['Weight', 'Speed']),
        ('Deadlift', 'Absolute Strength', ['Weight', 'Speed']),
        ('Grip', 'Arm Side', ['Weight']),
        ('Grip', 'Glove Side', ['Weight']),
        ('Jump', 'Broad', ['Distance']),
        ('Jump', 'Lateral Block Leg', ['Distance']),
        ('Jump', 'Lateral Load Leg', ['Distance']),
        ('Jump', 'Vertical', ['Vert']),
        ('Jump', 'Vertical Block Leg', ['Vert']),
        ('Jump', 'Vertical Load Leg', ['Vert']),
    ],
    'Velo Throws': [
        *[(t, w, ['Max Velo', 'Average Velo']) for t in ['Roll Ins', 'Double Plays', 'Turn and Burns', 'Pulldowns', 'Catchers Velo', 'Mound Velo']
          for w in ['3oz', '4oz', '5oz', '6oz', '7oz']]
    ],
    'Trackman Fastball': [
        ('Trackman Bullpen', 'Fastball', [
            'Max Velo', 'Average Velo', 'Max Spin', 'Average Spin',
            'IVB', 'HB', 'Extension', 'Rel Height (ft)', 'Rel Side (ft)',
            'Gyro', 'VAA', 'Total Strike %'
        ])
    ],
    'ArmCare': [
        ('ArmCare', 'Fresh Exam', [
            'Arm Score', 'Total Strength', 'Shoulder Balance', 'SVR',
            'IR ROM', 'ER ROM', 'Flexion ROM'
        ])
    ],
    'Hitting': [
        *[(t, sub, ['Average EV', 'Max EV', 'Max Distance']) for t, sub in [
            ('HitTrax', 'Tee'), ('HitTrax', 'Front Toss'), ('HitTrax', 'Machine/BP'), ('HitTrax', 'Live AB')]],
        *[(t, sub, ['Max Bat Speed', 'Peak Hand Speed']) for t, sub in [
            ('Blast Motion', 'Tee'), ('Blast Motion', 'Front Toss'), ('Blast Motion', 'Machine/BP'), ('Blast Motion', 'Live AB')]]
    ]
}

# Flattened metric_dict for percentiles
metric_dict = {}
for group in metric_groups.values():
    for test_type, sub_type, metrics in group:
        metric_dict.setdefault((test_type, sub_type), []).extend(metrics)

# Unit dictionary by metric
unit_dict = {
    # General
    'Weight': 'lbs',
    'Speed': 'm/s',
    'Distance': 'feet',
    'Vert': 'inches',
    'Height': 'inches',

    # Pitching
    'Max Velo': 'MPH',
    'Average Velo': 'MPH',
    'Max Spin': 'RPM',
    'Average Spin': 'RPM',
    'IVB': 'inches',
    'HB': 'inches',
    'Extension': 'ft',
    'Rel Height (ft)': 'ft',
    'Rel Side (ft)': 'ft',
    'Gyro': '°',
    'VAA': '°',
    'Total Strike %': '%',

    # Hitting
    'Max Bat Speed': 'MPH',
    'Peak Hand Speed': 'MPH',
    'Average EV': 'MPH',
    'Max EV': 'MPH',
    'Max Distance': 'ft',

    # ArmCare
    'Arm Score': '',
    'Total Strength': '',
    'Shoulder Balance': '',
    'SVR': '',
    'IR ROM': '°',
    'ER ROM': '°',
    'Flexion ROM': '°'
}

label_lookup = {
        "Back Squat - Strength-Speed - Weight": "VBT Back Squat - Weight",
        "Back Squat - Strength-Speed - Speed": "VBT Back Squat - Speed",
        "Bench Press - Strength-Speed - Weight": "VBT Bench Press - Weight",
        "Bench Press - Strength-Speed - Speed": "VBT Bench Press - Speed",
        "Deadlift - Strength-Speed - Weight": "VBT Deadlift - Weight",
        "Deadlift - Strength-Speed - Speed": "VBT Deadlift - Speed",
        "Grip - Arm Side - Weight": "Grip Arm Side",
        "Grip - Glove Side - Weight": "Grip Glove Side",
        "Jump - Broad - Distance": "Broad Jump Distance",
        "Jump - Lateral Block Leg - Distance": "Block Leg Lateral Jump",
        "Jump - Lateral Load Leg - Distance": "Load Leg Lateral Jump",
        "Jump - Vertical - Vert": "Vertical Jump Height",
        "Jump - Vertical Block Leg - Vert": "Block Leg Vertical Jump",
        "Jump - Vertical Load Leg - Vert": "Load Leg Vertical Jump",
        "Weigh-in - Weigh-in - Weight": "Weight",
        "Weigh-in - Height - Height": "Height",
        "Roll Ins - 3oz - Max Velo": "Roll Ins 3oz Max",
        "Roll Ins - 3oz - Average Velo": "Roll Ins 3oz Avg",
        "Roll Ins - 4oz - Max Velo": "Roll Ins 4oz Max",
        "Roll Ins - 4oz - Average Velo": "Roll Ins 4oz Avg",
        "Roll Ins - 5oz - Max Velo": "Roll Ins 5oz Max",
        "Roll Ins - 5oz - Average Velo": "Roll Ins 5oz Avg",
        "Roll Ins - 6oz - Max Velo": "Roll Ins 6oz Max",
        "Roll Ins - 6oz - Average Velo": "Roll Ins 6oz Avg",
        "Roll Ins - 7oz - Max Velo": "Roll Ins 7oz Max",
        "Roll Ins - 7oz - Average Velo": "Roll Ins 7oz Avg",
        "Double Plays - 3oz - Max Velo": "Double Plays 3oz Max",
        "Double Plays - 3oz - Average Velo": "Double Plays 3oz Avg",
        "Double Plays - 4oz - Max Velo": "Double Plays 4oz Max",
        "Double Plays - 4oz - Average Velo": "Double Plays 4oz Avg",
        "Double Plays - 5oz - Max Velo": "Double Plays 5oz Max",
        "Double Plays - 5oz - Average Velo": "Double Plays 5oz Avg",
        "Double Plays - 6oz - Max Velo": "Double Plays 6oz Max",
        "Double Plays - 6oz - Average Velo": "Double Plays 6oz Avg",
        "Double Plays - 7oz - Max Velo": "Double Plays 7oz Max",
        "Double Plays - 7oz - Average Velo": "Double Plays 7oz Avg",
        "Turn and Burns - 3oz - Max Velo": "Turn and Burns 3oz Max",
        "Turn and Burns - 3oz - Average Velo": "Turn and Burns 3oz Avg",
        "Turn and Burns - 4oz - Max Velo": "Turn and Burns 4oz Max",
        "Turn and Burns - 4oz - Average Velo": "Turn and Burns 4oz Avg",
        "Turn and Burns - 5oz - Max Velo": "Turn and Burns 5oz Max",
        "Turn and Burns - 5oz - Average Velo": "Turn and Burns 5oz Avg",
        "Turn and Burns - 6oz - Max Velo": "Turn and Burns 6oz Max",
        "Turn and Burns - 6oz - Average Velo": "Turn and Burns 6oz Avg",
        "Turn and Burns - 7oz - Max Velo": "Turn and Burns 7oz Max",
        "Turn and Burns - 7oz - Average Velo": "Turn and Burns 7oz Avg",
        "Pulldowns - 3oz - Max Velo": "Pulldowns 3oz Max",
        "Pulldowns - 3oz - Average Velo": "Pulldowns 3oz Avg",
        "Pulldowns - 4oz - Max Velo": "Pulldowns 4oz Max",
        "Pulldowns - 4oz - Average Velo": "Pulldowns 4oz Avg",
        "Pulldowns - 5oz - Max Velo": "Pulldowns 5oz Max",
        "Pulldowns - 5oz - Average Velo": "Pulldowns 5oz Avg",
        "Pulldowns - 6oz - Max Velo": "Pulldowns 6oz Max",
        "Pulldowns - 6oz - Average Velo": "Pulldowns 6oz Avg",
        "Pulldowns - 7oz - Max Velo": "Pulldowns 7oz Max",
        "Pulldowns - 7oz - Average Velo": "Pulldowns 7oz Avg",
        "Trackman Bullpen - Fastball - Max Velo": "Max Velo",
        "Trackman Bullpen - Fastball - Average Velo": "Average Velo",
        "Trackman Bullpen - Fastball - Max Spin": "Max Spin",
        "Trackman Bullpen - Fastball - Average Spin": "Average Spin",
        "Trackman Bullpen - Fastball - IVB": "Induced Vertical Break",
        "Trackman Bullpen - Fastball - HB": "Horizontal Break",
        "Trackman Bullpen - Fastball - Extension": "Extension",
        "Trackman Bullpen - Fastball - Rel Height (ft)": "Release Height",
        "Trackman Bullpen - Fastball - Rel Side (ft)": "Release Side",
        "Trackman Bullpen - Fastball - Gyro": "Gyro",
        "Trackman Bullpen - Fastball - VAA": "Vertical Approach Angle",
        "Trackman Bullpen - Fastball - Total Strike %": "Total Strike %",
        "ArmCare - Fresh Exam - Arm Score": "Arm Score",
        "ArmCare - Fresh Exam - Total Strength": "Total Strength",
        "ArmCare - Fresh Exam - Shoulder Balance": "Shoulder Balance",
        "ArmCare - Fresh Exam - SVR": "Stregth to Velo Ratio",
        "ArmCare - Fresh Exam - IR ROM": "Internal Rotation ROM",
        "ArmCare - Fresh Exam - ER ROM": "External Rotation ROM",
        "ArmCare - Fresh Exam - Flexion ROM": "Flexion ROM",
        "HitTrax - Front Toss - Average EV": "Average Exit Velo",
        "HitTrax - Front Toss - Max EV": "Max Exit Velo",
        "HitTrax - Front Toss - Max Distance": "Max Distance",
        "Blast Motion - Front Toss - Max Bat Speed": "Max Bat Speed",
        "Blast Motion - Front Toss - Peak Hand Speed": "Peak Hand Speed",
    }

def calculate_percentiles(df, athlete_id, metric_dict, verbose=False):
    """
    Calculate percentile rankings for all test numbers for each metric for an athlete compared to peers at the same level.
    """
    from scipy.stats import percentileofscore

    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        if verbose:
            print(f"No data for athlete ID {athlete_id}")
        return pd.DataFrame(), None, None

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[df['Level'] == athlete_level]

    records = []

    for (test_type, sub_type), metrics in metric_dict.items():
        for metric in metrics:
            subset = peer_data[
                (peer_data['Test Type'] == test_type) &
                (peer_data['Test Sub-Type'] == sub_type) &
                (peer_data[metric].notna())
            ].copy()

            subset[metric] = pd.to_numeric(subset[metric], errors='coerce')
            subset = subset.dropna(subset=[metric])
            if subset.empty:
                continue

            athlete_metric_data = athlete_data[
                (athlete_data['Test Type'] == test_type) &
                (athlete_data['Test Sub-Type'] == sub_type) &
                (athlete_data[metric].notna())
            ].copy()

            athlete_metric_data[metric] = pd.to_numeric(athlete_metric_data[metric], errors='coerce')
            athlete_metric_data = athlete_metric_data.dropna(subset=[metric])

            for _, row in athlete_metric_data.iterrows():
                test_number = row.get('Test Number', None)
                test_date = row.get('Date', None)
                test_value = row[metric]
                percentile = percentileofscore(subset[metric], test_value)

                records.append({
                    'Test Type': test_type,
                    'Sub-Type': sub_type,
                    'Metric': metric,
                    'Test Number': test_number,
                    'Date': test_date,
                    'Value': test_value,
                    'Percentile': percentile,
                    'Athlete ID': athlete_id,
                    'Level': athlete_level
                })

                if verbose:
                    print(f"{test_type} - {sub_type} - {metric} | Test #{test_number} | {test_date}: Value={test_value} → Percentile={percentile:.1f}%")

    results_df = pd.DataFrame(records)
    return results_df, athlete_name, athlete_level


def plot_savant_chart(percentile_df, athlete_name, pdf):
    from matplotlib.colors import LinearSegmentedColormap
    from datetime import datetime
    import matplotlib.patheffects as path_effects
    from matplotlib.patches import Rectangle

    print(f"[DEBUG] plot_savant_chart called for {athlete_name}")

    custom_cmap = LinearSegmentedColormap.from_list("BlueRed", ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"])
    current_date = datetime.now().strftime("%B %d, %Y")

    categories = metric_groups.keys()
    for category in categories:
        rows = []
        labels_in_order = []

        for test_type, sub_type, metrics in metric_groups[category]:
            if category == 'Strength' and test_type in ['Back Squat', 'Bench Press', 'Deadlift'] and sub_type != 'Strength-Speed':
                continue

            for metric in metrics:
                filtered = percentile_df[
                    (percentile_df['Test Type'] == test_type) &
                    (percentile_df['Sub-Type'] == sub_type) &
                    (percentile_df['Metric'] == metric)
                ]
                if not filtered.empty:
                    most_recent = filtered.sort_values(by='Test Number', ascending=False).iloc[0].copy()
                    full_label = f"{test_type} - {sub_type} - {metric}"
                    most_recent['Metric Label'] = full_label
                    most_recent['Display Label'] = label_lookup.get(full_label, full_label)
                    most_recent['Display Value'] = most_recent['Value']
                    most_recent['Display Percentile'] = most_recent['Percentile']
                    rows.append(most_recent)
                    labels_in_order.append(most_recent['Display Label'])

        if not rows:
            continue

        cat_df = pd.DataFrame(rows)
        cat_df.set_index("Display Label", inplace=True)
        cat_df = cat_df.loc[labels_in_order[:len(cat_df)]]

        percentiles = cat_df['Display Percentile']
        values = cat_df['Display Value']
        metrics = cat_df['Metric']
        y_labels = cat_df.index.tolist()

        bar_height = 0.36
        fig_height = bar_height * len(y_labels) + 2.5
        fig, ax = plt.subplots(figsize=(10, fig_height))
        ax.set_facecolor('white')

        norm = plt.Normalize(0, 100)
        bar_colors = custom_cmap(norm(percentiles))

        ax.barh(y_labels, [100]*len(y_labels), color='#eeeeee', height=bar_height*0.5, zorder=0)
        bars = ax.barh(y_labels, percentiles, color=bar_colors, height=bar_height, zorder=1)

        # Draw border
        border = Rectangle((0, 0), 1, 1, transform=ax.transAxes, linewidth=1.2, edgecolor='black', facecolor='none', zorder=10)
        ax.add_patch(border)

        for i, (bar, perc, val, met, label) in enumerate(zip(bars, percentiles, values, metrics, y_labels)):
            unit = unit_dict.get(met, '')
            bar_center = bar.get_y() + bar.get_height() / 2
            badge_color = bar_colors[i]

            # Label inside bar
            ax.text(
                2.5, bar_center, label,
                ha='left', va='center', fontsize=9, weight='bold', color='white',
                path_effects=[
                    path_effects.Stroke(linewidth=1.5, foreground='black', alpha=0.25),
                    path_effects.Normal()
                ],
                zorder=3
            )

            # Circle tag at the end
            ax.text(
                bar.get_width(), bar_center, f"{int(perc)}",
                ha='center', va='center', fontsize=8, color='white', fontweight='bold',
                bbox=dict(boxstyle='circle,pad=0.4', fc=badge_color, ec='white', linewidth=1.0, alpha=0.95),
                zorder=4
            )

            # Value pill on the right (tucked in before edge)
            ax.text(101, bar_center, f"{val:.2f} {unit}" if met == 'Speed' else f"{val:.1f} {unit}", va='center', ha='left', fontsize=8,
                    bbox=dict(facecolor='white', edgecolor='lightgray', boxstyle='round,pad=0.2'), zorder=2)

        # Header and watermark
        ax.text(0, 1.07, f"{category} — Performance Summary", fontsize=16, fontweight='bold', transform=ax.transAxes, ha='left')
        ax.text(1.0, 1.07, "Beimel Elite Athletics", transform=ax.transAxes, fontsize=10, ha='right', color='gray', alpha=0.7)
        ax.text(0.98, 0.02, "Metric", fontsize=36, color='gray', alpha=0.08, ha='right', va='bottom', transform=ax.transAxes)

        # Clean layout
        ax.set_xlim(0, 110)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.invert_yaxis()
        for spine in ax.spines.values():
            spine.set_visible(False)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()


def generate_scope_report(worksheet, athlete_id, save_directory):
    """
    Generate a sequential, modular SCOPE 2.0 progress report for an athlete.

    Args:
        worksheet: gspread worksheet object to pull data from.
        athlete_id (int): Athlete ID to generate report for.
        save_directory (str): Folder to save the PDF output.
    """
    import pandas as pd
    from matplotlib.backends.backend_pdf import PdfPages
    from datetime import datetime
    import os

    # Load data from worksheet
    df = pd.DataFrame(worksheet.get_all_records())
    print("[DEBUG] Data loaded from worksheet")

    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    # Get name
    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

    # Get most recent strength workout phase
    strength_data = athlete_data[athlete_data['Workout Type'] == 'Strength']
    if not strength_data.empty:
        strength_data_sorted = strength_data.sort_values(by=['Test Number', 'Date'], ascending=[False, False])
        recent_strength = strength_data_sorted.iloc[0]
        phase_name = recent_strength.get('Phase', '')
    else:
        phase_name = ''

    # Build file name and path
    today = datetime.now().strftime("%Y-%m-%d")
    file_name = f"{athlete_name}, {phase_name}, {today}.pdf"
    pdf_path = os.path.join(save_directory, file_name)
    print(f"[DEBUG] Output path set to: {pdf_path}")

    # ➕ Calculate Percentiles
    percentile_df, athlete_name, athlete_level = calculate_percentiles(df, athlete_id, metric_dict)
    print(f"[DEBUG] Percentile DF shape: {percentile_df.shape}")
    print("[DEBUG] Sample rows from percentile_df:")
    print(percentile_df[['Test Type', 'Sub-Type', 'Metric', 'Value', 'Percentile']].head(5))
    print("[DEBUG] Unique metrics:")
    print(percentile_df['Metric'].dropna().unique())

    try:
        with PdfPages(pdf_path) as pdf:
            print(f"[DEBUG] PDF file opened at {pdf_path}")

            print("[DEBUG] Adding cover page...")
            generate_cover_page(df, athlete_id, pdf)

            print("[DEBUG] Adding Savant chart...")
            plot_savant_chart(percentile_df, athlete_name, pdf)

        print(f"SCOPE 2.0 report saved to: {pdf_path}")

    except Exception as e:
        print(f"[ERROR] Failed to save PDF: {e}")


    save_directory='C:/Users/benoi/OneDrive/Desktop/bea/SCOPE_Reports'


In [60]:
# -----------------------------------------------
# 🔧 Load Data from Google Sheets
# -----------------------------------------------
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials_path = "C:/Users/benoi/OneDrive/Desktop/bea/json credentials/bea-data-7dda3770b44f.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)

client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OfgrNbgZjwMMAuVdIhOLywXUHEw1xHzYaN8ZU2awsSs/edit?gid=1169755047')
worksheet = sheet.get_worksheet(0)

# -----------------------------------------------
# ✅ Manually Specify Report Target
# -----------------------------------------------
athlete_id = 112109
save_directory = "C:/Users/benoi/OneDrive/Desktop/bea/SCOPE_Reports"

print("[DEBUG] About to call generate_scope_report...")

# -----------------------------------------------
# 🚀 Generate the Report
# -----------------------------------------------
generate_scope_report(worksheet, athlete_id, save_directory)


[DEBUG] About to call generate_scope_report...
[DEBUG] Data loaded from worksheet
[DEBUG] Output path set to: C:/Users/benoi/OneDrive/Desktop/bea/SCOPE_Reports\Matthew Coye, Phase 3, 2025-04-14.pdf
[DEBUG] Percentile DF shape: (314, 9)
[DEBUG] Sample rows from percentile_df:
  Test Type  Sub-Type  Metric  Value  Percentile
0  Weigh-in    Height  Height   68.0   29.166667
1  Weigh-in    Height  Height   69.0   41.666667
2  Weigh-in  Weigh-in  Weight  145.0   24.429224
3  Weigh-in  Weigh-in  Weight  146.1   26.027397
4  Weigh-in  Weigh-in  Weight  149.0   28.538813
[DEBUG] Unique metrics:
['Height' 'Weight' 'Speed' 'Distance' 'Vert' 'Max Velo' 'Average Velo'
 'Max Spin' 'Average Spin' 'IVB' 'HB' 'Extension' 'Rel Height (ft)'
 'Rel Side (ft)' 'Gyro' 'VAA' 'Total Strike %' 'Arm Score'
 'Total Strength' 'Shoulder Balance' 'SVR' 'IR ROM' 'ER ROM' 'Flexion ROM'
 'Average EV' 'Max EV' 'Max Distance' 'Max Bat Speed' 'Peak Hand Speed']
[DEBUG] PDF file opened at C:/Users/benoi/OneDrive/Desktop/b

    label_lookup = {
        "Back Squat - Strength-Speed - Weight": "Back Squat Weight",
        "Back Squat - Strength-Speed - Speed": "Back Squat Speed",
        "Bench Press - Strength-Speed - Weight": "Bench Press Weight",
        "Bench Press - Strength-Speed - Speed": "Bench Press Speed",
        "Deadlift - Strength-Speed - Weight": "Deadlift Weight",
        "Deadlift - Strength-Speed - Speed": "Deadlift Speed",
        "Grip - Arm Side - Weight": "Grip Arm Side",
        "Grip - Glove Side - Weight": "Grip Glove Side",
        "Jump - Broad - Distance": "Broad Jump",
        "Jump - Lateral Block Leg - Distance": "Block Leg Lateral",
        "Jump - Lateral Load Leg - Distance": "Load Leg Lateral",
        "Jump - Vertical - Vert": "Vertical Jump",
        "Jump - Vertical Block Leg - Vert": "Block Leg Vertical",
        "Jump - Vertical Load Leg - Vert": "Load Leg Vertical",
        "Weigh-in - Weigh-in - Weight": "Weight",
        "Weigh-in - Height - Height": "Height",
        "Roll Ins - 3oz - Max Velo": "Roll Ins 3oz Max",
        "Roll Ins - 3oz - Average Velo": "Roll Ins 3oz Avg",
        "Roll Ins - 4oz - Max Velo": "Roll Ins 4oz Max",
        "Roll Ins - 4oz - Average Velo": "Roll Ins 4oz Avg",
        "Roll Ins - 5oz - Max Velo": "Roll Ins 5oz Max",
        "Roll Ins - 5oz - Average Velo": "Roll Ins 5oz Avg",
        "Roll Ins - 6oz - Max Velo": "Roll Ins 6oz Max",
        "Roll Ins - 6oz - Average Velo": "Roll Ins 6oz Avg",
        "Roll Ins - 7oz - Max Velo": "Roll Ins 7oz Max",
        "Roll Ins - 7oz - Average Velo": "Roll Ins 7oz Avg",
        "Double Plays - 3oz - Max Velo": "Double Plays 3oz Max",
        "Double Plays - 3oz - Average Velo": "Double Plays 3oz Avg",
        "Double Plays - 4oz - Max Velo": "Double Plays 4oz Max",
        "Double Plays - 4oz - Average Velo": "Double Plays 4oz Avg",
        "Double Plays - 5oz - Max Velo": "Double Plays 5oz Max",
        "Double Plays - 5oz - Average Velo": "Double Plays 5oz Avg",
        "Double Plays - 6oz - Max Velo": "Double Plays 6oz Max",
        "Double Plays - 6oz - Average Velo": "Double Plays 6oz Avg",
        "Double Plays - 7oz - Max Velo": "Double Plays 7oz Max",
        "Double Plays - 7oz - Average Velo": "Double Plays 7oz Avg",
        "Turn and Burns - 3oz - Max Velo": "Turn and Burns 3oz Max",
        "Turn and Burns - 3oz - Average Velo": "Turn and Burns 3oz Avg",
        "Turn and Burns - 4oz - Max Velo": "Turn and Burns 4oz Max",
        "Turn and Burns - 4oz - Average Velo": "Turn and Burns 4oz Avg",
        "Turn and Burns - 5oz - Max Velo": "Turn and Burns 5oz Max",
        "Turn and Burns - 5oz - Average Velo": "Turn and Burns 5oz Avg",
        "Turn and Burns - 6oz - Max Velo": "Turn and Burns 6oz Max",
        "Turn and Burns - 6oz - Average Velo": "Turn and Burns 6oz Avg",
        "Turn and Burns - 7oz - Max Velo": "Turn and Burns 7oz Max",
        "Turn and Burns - 7oz - Average Velo": "Turn and Burns 7oz Avg",
        "Pulldowns - 3oz - Max Velo": "Pulldowns 3oz Max",
        "Pulldowns - 3oz - Average Velo": "Pulldowns 3oz Avg",
        "Pulldowns - 4oz - Max Velo": "Pulldowns 4oz Max",
        "Pulldowns - 4oz - Average Velo": "Pulldowns 4oz Avg",
        "Pulldowns - 5oz - Max Velo": "Pulldowns 5oz Max",
        "Pulldowns - 5oz - Average Velo": "Pulldowns 5oz Avg",
        "Pulldowns - 6oz - Max Velo": "Pulldowns 6oz Max",
        "Pulldowns - 6oz - Average Velo": "Pulldowns 6oz Avg",
        "Pulldowns - 7oz - Max Velo": "Pulldowns 7oz Max",
        "Pulldowns - 7oz - Average Velo": "Pulldowns 7oz Avg",
        "Trackman Bullpen - Fastball - Max Velo": "Max Velo",
        "Trackman Bullpen - Fastball - Average Velo": "Average Velo",
        "Trackman Bullpen - Fastball - Max Spin": "Max Spin",
        "Trackman Bullpen - Fastball - Average Spin": "Average Spin",
        "Trackman Bullpen - Fastball - IVB": "Induced Vertical Break",
        "Trackman Bullpen - Fastball - HB": "Horizontal Break",
        "Trackman Bullpen - Fastball - Extension": "Extension",
        "Trackman Bullpen - Fastball - Rel Height (ft)": "Release Height",
        "Trackman Bullpen - Fastball - Rel Side (ft)": "Release Side",
        "Trackman Bullpen - Fastball - Gyro": "Gyro",
        "Trackman Bullpen - Fastball - VAA": "Vertical Approach Angle",
        "Trackman Bullpen - Fastball - Total Strike %": "Total Strike %",
        "ArmCare - Fresh Exam - Arm Score": "Arm Score",
        "ArmCare - Fresh Exam - Total Strength": "Total Strength",
        "ArmCare - Fresh Exam - Shoulder Balance": "Shoulder Balance",
        "ArmCare - Fresh Exam - SVR": "Stregth to Velo Ratio",
        "ArmCare - Fresh Exam - IR ROM": "Internal Rotation ROM",
        "ArmCare - Fresh Exam - ER ROM": "External ROM",
        "ArmCare - Fresh Exam - Flexion ROM": "Flexion ROM",
        "HitTrax - Front Toss - Average EV": "Average Exit Velo",
        "HitTrax - Front Toss - Max EV": "Max Exit Velo",
        "HitTrax - Front Toss - Max Distance": "Max Distance",
        "Blast Motion - Front Toss - Max Bat Speed": "Max Bat Speed",
        "Blast Motion - Front Toss - Peak Hand Speed": "Peak Hand Speed",
    }
